# Problem 4.6

## part a

In [2]:
clear
qui import delim  ./data/chapter04/OBrien_Patsiorkovski_ICPSR_1999,clear

qui recode sexresp6 ( 2 = 1) (1 = 0), gen(female)
qui recode memberc6 ( 1 = 1) (else = 0), gen(church_member)

In [3]:
// fit1
qui reg index96 index95 i.female i.church_member i.group6
disp %8.4f e(r2)



  0.3937


In [4]:
// fit2
qui reg index96 i.female i.church_member i.group6
disp %8.5f e(r2)



 0.02828


## part b

In [8]:
//ssc install randtreat (install the package for random assigment)
qui randtreat, generate(treat) strata(female church_member index96 group6) misfits(wglobal)

In [9]:
// coefficient is different due to the nature of random assignment
// but the R-squred is similar 0.00, meaning the balance hold on 
// the covariates
regress treat female church_member i.group6 index96


      Source |       SS           df       MS      Number of obs   =       462
-------------+----------------------------------   F(6, 455)       =      0.03
       Model |  .042679933         6  .007113322   Prob > F        =    0.9999
    Residual |   115.45732       455  .253752352   R-squared       =    0.0004
-------------+----------------------------------   Adj R-squared   =   -0.0128
       Total |       115.5       461  .250542299   Root MSE        =    .50374

-------------------------------------------------------------------------------
        treat |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
--------------+----------------------------------------------------------------
       female |  -.0075749   .0552815    -0.14   0.891    -.1162136    .1010638
church_member |   .0134376   .0817447     0.16   0.870    -.1472064    .1740816
              |
       group6 |
           2  |  -.0484915   .1590765    -0.30   0.761    -.3611072    .2641243
           3

## part d

In [12]:
set seed 1234567
// complete assignment
clear*

capture program drop com_ra
program define com_ra, rclass
import delim  ./data/chapter04/OBrien_Patsiorkovski_ICPSR_1999,clear
capture drop complete
randtreat, generate(complete)
regress index97 complete
return scalar coef = _b[complete]
exit
end

qui simulate  coef=r(coef), reps(10000): com_ra
qui sum
scalar complete_est = r(mean)
scalar complete_sd = r(sd)

di "Average Estimate (complete)" %8.2f complete_est
di "Standard Error (complete)"%8.2f complete_sd










Average Estimate (complete)    0.00

Standard Error (complete)    0.17


In [13]:
// adjusted
capture program drop adj_ra
program define adj_ra, rclass
import delim  ./data/chapter04/OBrien_Patsiorkovski_ICPSR_1999,clear
capture drop complete
randtreat, generate(complete)
regress index97 complete sexresp6 memberc6 i.group6 index96
return scalar coef = _b[complete]
exit
end


qui simulate  coef=r(coef), reps(10000): adj_ra
qui sum
scalar adjusted_est = r(mean)
scalar adjusted_sd = r(sd)

di "Average Estimate (adjusted)"%8.2f adjusted_est
di "Standard Error (adjusted)"%8.2f adjusted_sd









Average Estimate (adjusted)   -0.00

Standard Error (adjusted)    0.13


In [14]:
// blocked
capture program drop block_ra
program define block_ra, rclass
import delim  ./data/chapter04/OBrien_Patsiorkovski_ICPSR_1999,clear
capture drop block
randtreat, generate(block) strata(sexresp6 memberc6 group6 index96) misfits(wglobal)
regress index97 block
return scalar coef = _b[block]
exit
end


qui simulate  coef=r(coef), reps(10000): block_ra
qui sum
scalar block_est = r(mean)
scalar block_sd = r(sd)

di "Average Estimate (blocked)"%8.2f block_est
di "Standard Error (blocked)"%8.2f block_sd








Average Estimate (blocked)    0.00

Standard Error (blocked)    0.12


In [15]:
// pool results together
disp %8.2f complete_est %8.2f adjusted_est %8.2f block_est
disp %8.2f complete_sd %8.2f adjusted_sd %8.2f block_sd


    0.00   -0.00    0.00

    0.17    0.13    0.12
